# 🔐 About the Encrypto-Anonymizer

- Uploaded files are stored **temporarily** on the virtual machine running your Colab session.
- Only **you** (the logged-in Google account) have access to them during that session.
- Files are **not shared** with Google, the public, or other users unless you explicitly do so.
- Colab does **not store** those files persistently — they are deleted when the session ends (e.g., if idle for 90 minutes or closed).


## 📁 Upload a file to anonymize

This tool lets you anonymize and encrypt CSV files for safe review, processing, and later re-identification.

In [ ]:
#@title Upload your CSV
from google.colab import files
uploaded = files.upload()
